<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250723/%E3%82%AB%E3%83%83%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E3%82%B9%E3%83%88%E3%83%83%E3%82%AF%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#準備

In [ ]:
%%capture
# インストール
!pip install mip
!pip install japanize-matplotlib

# カッティングストック問題


本講義では「カッティングストック問題 (Cutting Stock Problem, CSP)」を扱う。
この問題は、限られた大きさの原材料から、顧客の要求する様々なサイズの製品を切り出す際に、原材料の無駄を最小限に抑える、あるいは使用する原材料の総量を最小化することを目的とします。製紙、鉄鋼、繊維、ガラスなど、多くの産業でコスト削減や資源効率の向上に直結する重要な課題です。

本講義の目標は以下の通りです。

1.  1次元カッティングストック問題の基本的な概念と、その産業界での応用を理解する。
2.  カッティングストック問題を数理モデル（特にパターンベースのアプローチ）として定式化する手法を習得する。
3.  Pythonの最適化ライブラリ `python-mip` を用いて、カッティングストック問題を実際に解くプログラミングスキルを身につける。
4.  提示された課題を通じて、問題解決能力と応用力を養う。


## カッティングストック問題とは

### 問題の定義 (1次元カッティングストック問題)

1次元カッティングストック問題は、標準サイズの原反（例えば、ロール状の紙や鋼板、一定の長さの棒材など）から、複数の異なる長さ（または幅）の製品を、それぞれの製品の要求数を満たすように切り出す際に、使用する原反の総本数を最小化する問題である。このとき、切り出し方によっては原反に「トリムロス」と呼ばれる未使用部分（無駄）が生じるため、これを減らすことも重要な目的となる。

  * **入力**:
      * 原反の標準サイズ $L$（例: 幅100cmのロール紙）
      * 製品の種類の集合 $I = {0, 1, \\dots, m-1}$
      * 各製品 $i \\in I$ のサイズ $w\_i$（例: 製品Aは幅30cm）
      * 各製品 $i \\in I$ の需要量 $D\_i$（例: 製品Aは10本必要）
  * **出力**:
      * 各原反からどの製品を何本ずつ切り出すかの「カッティングパターン」の組み合わせ
      * 各パターンを何本の原反に適用するかの決定
  * **目的**:
      * 全ての製品の需要量を満たしつつ、使用する原反の総本数を最小化する。

この問題は、組み合わせ最適化問題の一種であり、製品の種類や需要量が増えると最適な解を見つけるのが難しくなるNP困難な問題として知られている。
また、製品のサイズとして1方向のみを考慮することから
**1次元**カッティングストック問題と呼ばれる。

### 簡単な例題

具体的な例で考えてみよう。

  * 原反の幅（長さ）: $L = 100 \text{cm}$
  * 注文 (製品):
      * 製品A: 幅（長さ） $w\_A = 30 \\text{cm}$, 需要 $D\_A = 5 \\text{本}$
      * 製品B: 幅（長さ） $w\_B = 45 \\text{cm}$, 需要 $D\_B = 5 \\text{本}$
      * 製品C: 幅（長さ） $w\_C = 55 \\text{cm}$, 需要 $D\_C = 3 \\text{本}$

これらの注文を満たすために、幅100cmの原反は最小何本必要だとうか。

例えば、1本の原反から以下のように切り出すことができる（これを「カッティングパターン」と呼ぶ）。

  * **パターン0**: 製品Aを3本 (30cm $\times$ 3 = 90cm)。残り10cmはトリムロス。
  * **パターン1**: 製品Bを2本 (45cm $\times$ 2 = 90cm)。残り10cmはトリムロス。
  * **パターン2**: 製品Cを1本、製品Aを1本 (55cm + 30cm = 85cm)。残り15cmはトリムロス。
  * **パターン3**: 製品Cを1本、製品Bを1本 (55cm + 45cm = 100cm)。残り0cm。

これらのパターンをうまく組み合わせて、全ての需要を満たし、かつ使用する原反の本数を最小にすることを考える。


####例題の数理モデル

カッティングパターン $j$ を使用する回数（原反の本数）を決定変数とし$x_j$ として、
数理モデルを考える。

目的関数は原反の総数なので
$$
x_0 + x_1 + x_2 + x_3
$$
である。

変数とパターンとの関係は、以下の表となる。

| 変数 | パターンID | A (30cm)  | B (45cm)  | C (55cm)  | 合計幅 | トリムロス |
| ---  |------------|-----------|-----------|-----------|--------|------------|
|$x_0$ | 0          | 3         | 0         | 0         | 90     | 10         |
|$x_1$ | 1          | 0         | 2         | 0         | 90     | 10         |
|$x_3$ | 2          | 1         | 0         | 1         | 85     | 15         |
|$x_4$ | 3          | 0         | 1         | 1         | 100    | 0          |

この時、製品Aの個数が需要量（5個）以上となるためには、
変数 $x_j$ は以下の制約条件
$$
 3 x_0 + 0 x_2 + 1 x_3 + 0 x_4 \ge 5
$$
を満たさなくてはならない。
同様に、製品B、Cに関しても制約条件として
$$
 0 x_0 + 2 x_2 + 0 x_3 + 1 x_4 \ge 5
$$
$$
 0 x_0 + 0 x_2 + 1 x_3 + 1 x_4 \ge 3
$$
となる。

これは、目的関数も制約条件も線形なので、整数計画問題となる。

### 実社会での応用例

カッティングストック問題は、多くの産業で資源効率を最大化するために不可欠である。

  * **製紙業**: 大きなロール紙から、新聞、書籍、包装用紙など、様々な幅や長さの紙製品を切り出す。
  * **製鉄・金属加工業**: 標準サイズの鋼板や棒鋼から、自動車部品、建築資材などを切り出す。
  * **繊維産業**: 布地のロールから、衣類パーツを裁断する。
  * **ガラス産業**: 大きなガラス板から、窓ガラスや鏡などを切り出す。
  * **木材加工業**: 標準的な長さの木材から、家具の部品や建材を切り出す。
  * **パイプライン建設**: 定尺のパイプから必要な長さのパイプを切り出す。

これらの応用では、トリムロスを数パーセント削減するだけでも、年間で莫大なコスト削減に繋がることがある。

## 数理モデルによる定式化 (パターンベース・アプローチ)

カッティングストック問題を解くためのアプローチの一つに、「パターンベース・アプローチ」がある。これは、まず1本の原反から製品を切り出す様々な方法（カッティングパターン）を考え、次にこれらのパターンを何回ずつ使えば需要を満たしつつ原反使用数を最小化できるか、という問題として定式化するものである。

### カッティングパターン

カッティングパターンとは、1本の原反から各種類の製品を何本ずつ切り出すかの具体的な組み合わせである。
例えば、原反の長さが $L$ で、製品 $i$ の長さが $w_i$ のとき、
あるパターン $j$ で製品 $i$ が $a_{ij}$ 本切りだされるとする。
このとき、製品として使われない（通常は廃棄あるいは原反に再利用）部分の長さ
$$
L - \sum_{i} a_{ij} w_i
$$
をトリムロスと呼ぶ。

**重要な注意点:** 実際の問題では、考えられる有効なカッティングパターンの数は膨大になることがある。そのため、全てのパターンを事前に列挙することは困難な場合が多い。本講義では、**比較的小規模な問題や、事前にいくつかの「妥当な」カッティングパターンが与えられている、あるいは専門家が知っている主要なパターンに限定されている状況を仮定する。** この仮定のもとで、どのパターンを何回使用するかを決定する問題を解く。

Gilmore-Gomory法のような高度な手法では、有望なパターンを最適化の過程で動的に生成する（遅延列生成法）方法も知られているが、
本講義の範囲を超える内容となるため、ここでは説明を行わない。

### 決定変数

  * $x_j$: カッティングパターン $j$ を使用する回数（すなわち、パターン $j$ でカットされる原反の本数）。
  $x_j$ は非負の整数でなければならない。

  * $j \in \{0, 1, \dots, P-1\}$ （$P$ は事前に定義されたカッティングパターンの総数）

### パラメータ

  * $L$: 原反のサイズ（長さまたは幅）
  * $m$: 製品の種類の数
  * $D_i$: 製品 $i$ の需要量 ($i \in \{0, \dots, m-1\}$)
  * $w_i$: 製品 $i$ のサイズ ($i \in \{0, \dots, m-1\}$)
  * $P$: 事前に定義されたカッティングパターンの総数
  * $a_{ij}$: 事前に定義されたカッティングパターン $j$ を用いた場合に、製品 $i$ が切り出される本数

### 目的関数

目的は、使用する原反の総本数を最小化することである。これは、各パターンの使用回数の合計で表される。

$$\min \sum_{j=0}^{P-1} x_j$$

### 制約条件

1.  **各製品の需要充足制約**:
    各製品 $i$ について、全てのパターンを通じて生産される総本数が、その製品の需要量 $D\_i$ 以上でなければならない。
    $$\sum_{j=0}^{P-1} a_{ij} x_j \ge D_i \quad \forall i \in \{0, 1, \dots, m-1\}$$

2.  **変数の型制約**:
    各パターンの使用回数 $x\_j$ は非負の整数でなければならない。
    $$x_j \ge 0 \quad \text{and integer} \quad \forall j \in \{0, 1, \dots, P-1\}$$

### 数理モデルのまとめ (パターンベース)

**目的関数**:
$$\min \sum_{j=0}^{P-1} x_j$$

**制約条件**:
$$\sum_{j=0}^{P-1} a_{ij} x_j \ge D_i \quad \forall i \in \{0, 1, \dots, m-1\} \quad \text{(需要充足制約)}$$
$$x_j \in \{0, 1, 2, \dots\} \quad \forall j \in \{0, 1, \dots, P-1\} \quad \text{(変数型制約)}$$

この定式化は、適切なカッティングパターン $a_{ij}$ が与えられていれば、整数計画問題として解くことができる。

## Python MIP を用いた実装

ここでは、`python-mip` を用いて、上記のパターンベースのカッティングストック問題を解く方法を学ぶ。

まず、必要なライブラリをインポートする。

In [ ]:
# 必要なライブラリをインポートする
import mip
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # 結果表示のため

### 問題データの設定 (例題)

以下のデータでカッティングストック問題を解く。

  * 原反の幅: $L = 100 \text{cm}$

  * 製品の種類と需要:

      * 製品A: 幅 $w\_0 = 45 \\text{cm}$, 需要 $D\_0 = 10 \\text{本}$
      * 製品B: 幅 $w\_1 = 35 \\text{cm}$, 需要 $D\_1 = 12 \\text{本}$
      * 製品C: 幅 $w\_2 = 25 \\text{cm}$, 需要 $D\_2 = 8 \\text{本}$

  * 製品のインデックスと名前のマッピング:

```
product_names = ["A (45cm)", "B (35cm)", "C (25cm)"]`
product_widths = [45, 35, 25]
demands = [10, 12, 8]
```

  * **事前に定義されたカッティングパターン**:
    各パターンは、1本の原反から各製品 (A, B, C) を何本切り出すかを示す。
    (パターン内の製品配置はここでは問わない)

| パターンID | A (45cm)  | B (35cm)  | C (25cm)  | 合計幅 | トリムロス |
|------------|-----------|-----------|-----------|--------|------------|
| 0          | 2         | 0         | 0         | 90     | 10         |
| 1          | 1         | 1         | 0         | 80     | 20         |
| 2          | 1         | 0         | 2         | 95     | 5          |
| 3          | 0         | 2         | 0         | 70     | 30         |
| 4          | 0         | 1         | 2         | 85     | 15         |
| 5          | 0         | 0         | 4         | 100    | 0          |
| 6          | 0         | 2         | 1         | 95     | 5          |
| 7          | 1         | 0         | 0         | 45     | 55         |

ここで、$a_{ij}$ を、パターン $j$ で製品 $i$ が取れる本数とする。
また、カッティングパターン全体を、
パターンを行、製品を列とする行列として表現することとする。


In [ ]:
# 原反の幅
stock_width = 100

# 製品データ
product_names = ["A (45cm)", "B (35cm)", "C (25cm)"]
product_widths = np.array([45, 35, 25])
demands = np.array([10, 12, 8])
num_product_types = len(product_names)

# 事前に定義されたカッティングパターン (pattern_yields[j][i]: パターンjで製品iが取れる本数)
# 各行が1つのパターンを表し、列が製品0, 製品1, 製品2の取得本数を示す
defined_patterns = [
    # A, B, C
    [2, 0, 0],  # Pattern 0
    [1, 1, 0],  # Pattern 1
    [1, 0, 2],  # Pattern 2
    [0, 2, 0],  # Pattern 3
    [0, 1, 2],  # Pattern 4
    [0, 0, 4],  # Pattern 5
    [0, 2, 1],  # Pattern 6
    [1, 0, 0],  # Pattern 7
]
pattern_yields = np.array(defined_patterns)
num_defined_patterns = len(defined_patterns)

# 各パターンの使用幅とトリムロスを計算（情報として）
pattern_total_widths = np.dot(pattern_yields, product_widths)
pattern_trim_losses = stock_width - pattern_total_widths

print(f"原反の幅: {stock_width} cm")
print(f"製品の種類数: {num_product_types}")
print(f"製品名: {product_names}")
print(f"製品幅: {product_widths}")
print(f"製品需要: {demands}")
print(f"\n定義されたカッティングパターンの数: {num_defined_patterns}")
print("パターンごとの製品取得本数 (行:パターン, 列:製品):")
print(pattern_yields)
print("\nパターンごとの使用幅:")
print(pattern_total_widths)
print("パターンごとのトリムロス:")
print(pattern_trim_losses)

# パターンが原反幅を超えていないかチェック
if np.any(pattern_total_widths > stock_width):
    print("\n警告: 原反幅を超える無効なパターンが含まれています！")
    for j, width in enumerate(pattern_total_widths):
        if width > stock_width:
            print(f"  パターン {j}: 幅 {width} > 原反幅 {stock_width}")

原反の幅: 100 cm
製品の種類数: 3
製品名: ['A (45cm)', 'B (35cm)', 'C (25cm)']
製品幅: [45 35 25]
製品需要: [10 12  8]

定義されたカッティングパターンの数: 8
パターンごとの製品取得本数 (行:パターン, 列:製品):
[[2 0 0]
 [1 1 0]
 [1 0 2]
 [0 2 0]
 [0 1 2]
 [0 0 4]
 [0 2 1]
 [1 0 0]]

パターンごとの使用幅:
[ 90  80  95  70  85 100  95  45]
パターンごとのトリムロス:
[10 20  5 30 15  0  5 55]


### モデルの作成と変数の定義

`python-mip` を用いてモデルを構築する。

1.  `Model` オブジェクトを作成する。目的は最小化 (`sense=mip.MINIMIZE`)。
2.  決定変数 $x_j$ (各パターンの使用回数) を定義する。これは非負の整数変数 (`var_type=mip.INTEGER`, `lb=0`) である。

In [ ]:
# 1. モデルの作成
model_csp = mip.Model(name="cutting_stock", sense=mip.MINIMIZE, solver_name=mip.CBC)

# 2. 変数の定義
# x_j: パターン j を使用する回数 (原反の本数)
x = [model_csp.add_var(var_type=mip.INTEGER, lb=0, name=f"x_{j}") for j in range(num_defined_patterns)]

### 目的関数の設定

モデルの目的関数は、使用する原反の総本数 $\sum x_j$ を最小化することである。

In [ ]:
# 3. 目的関数の設定
model_csp.objective = mip.xsum(x[j] for j in range(num_defined_patterns))

### 制約条件の追加

次に、定式化した制約条件をモデルに追加する。

1.  **各製品の需要充足制約**:
$$\sum_{j=0}^{P-1} a_{ij} x_j \ge D_i \quad \forall i \in \{0, 1, \dots, m-1\}$$

ここで $a_{ij}$ は、`pattern_yields[j][i]` (パターン $j$ で製品 $i$ が得られる本数) に対応する。


In [ ]:
# 4. 制約条件の追加
# 需要充足制約
for i in range(num_product_types): # 各製品タイプについて
    model_csp += mip.xsum(pattern_yields[j][i] * x[j] for j in range(num_defined_patterns)) >= demands[i], f"demand_prod_{i}"

### 問題の求解と結果の表示

モデルの定義が完了したら、ソルバーを実行して最適解を求める。求解結果（ステータス、目的関数値、変数の値など）を確認する。

In [ ]:
# 5. 問題の求解
print("カッティングストック問題の最適化を開始します...")
status_csp = model_csp.optimize()

# 6. 結果の表示
if status_csp == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました！")
    print(f"使用する原反の最小総本数: {int(model_csp.objective_value)}")

    print("\n各パターンの使用回数:")
    results_list = []
    for j in range(num_defined_patterns):
        if x[j].x > 1e-6: # ほぼ0より大きい場合 (浮動小数点誤差を考慮)
            print(f"  パターン {j} (製品構成: {pattern_yields[j]}): {int(round(x[j].x))} 回 (本)")
            results_list.append({
                "Pattern_ID": j,
                "A ": pattern_yields[j][0],
                "B ": pattern_yields[j][1],
                "C ": pattern_yields[j][2],
                "Times_Used": int(round(x[j].x)),
                "Trim_Loss_per_Stock": pattern_trim_losses[j]
            })

    results_df = pd.DataFrame(results_list)
    if not results_df.empty:
        print("\n使用されたパターンの詳細:")
        print(results_df.to_string(index=False))

    # 総生産量の確認
    total_production = np.zeros(num_product_types)
    for j in range(num_defined_patterns):
        if x[j].x > 1e-6:
            total_production += pattern_yields[j] * int(round(x[j].x))

    print("\n各製品の総生産量と需要量:")
    for i in range(num_product_types):
        print(f"  {product_names[i]}: 生産量 {int(total_production[i])}, 需要量 {demands[i]} (超過生産: {int(total_production[i] - demands[i])})")

    # 総トリムロスの計算
    total_trim_loss = 0
    for j in range(num_defined_patterns):
        if x[j].x > 1e-6:
            total_trim_loss += pattern_trim_losses[j] * int(round(x[j].x))
    total_stock_material_used = model_csp.objective_value * stock_width
    total_product_material_produced = np.sum(total_production * product_widths)
    print(f"パターンごとのロスから計算した総トリムロス（需要分のみ考慮した場合のロス）: {total_trim_loss}")
    if total_stock_material_used > 0:
        print(f"総トリムロス率 (対使用原反総材料): {total_trim_loss / total_stock_material_used * 100:.2f}%")


    # # 可視化: 使用されたパターンの回数
    # if not results_df.empty:
    #     plt.figure(figsize=(10, 6))
    #     active_pattern_labels = [f"Pattern {idx}" for idx in results_df["Pattern_ID"]]
    #     sns.barplot(x="Pattern_ID", y="Times_Used", data=results_df, palette="viridis", hue="Pattern_ID", dodge=False, legend=False) # legend=Falseを追加
    #     plt.title("Usage Count of Each Cutting Pattern")
    #     plt.xlabel("Pattern ID")
    #     plt.ylabel("Number of Times Used (Stock Rolls)")
    #     plt.xticks(ticks=range(len(results_df["Pattern_ID"])), labels=active_pattern_labels, rotation=45, ha="right")
    #     plt.tight_layout()
    #     plt.show()

elif status_csp == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解が存在しません。需要を満たすパターンがないか、制約が厳しすぎる可能性があります。")
elif status_csp == mip.OptimizationStatus.NO_SOLUTION_FOUND:
    print("解が見つかりませんでした（時間制限など）。")
else:
    print(f"最適化ステータス: {status_csp}")

カッティングストック問題の最適化を開始します...
最適解が見つかりました！
使用する原反の最小総本数: 12

各パターンの使用回数:
  パターン 0 (製品構成: [2 0 0]): 5 回 (本)
  パターン 2 (製品構成: [1 0 2]): 1 回 (本)
  パターン 6 (製品構成: [0 2 1]): 6 回 (本)

使用されたパターンの詳細:
 Pattern_ID  A   B   C   Times_Used  Trim_Loss_per_Stock
          0   2   0   0           5                   10
          2   1   0   2           1                    5
          6   0   2   1           6                    5

各製品の総生産量と需要量:
  A (45cm): 生産量 11, 需要量 10 (超過生産: 1)
  B (35cm): 生産量 12, 需要量 12 (超過生産: 0)
  C (25cm): 生産量 8, 需要量 8 (超過生産: 0)
パターンごとのロスから計算した総トリムロス（需要分のみ考慮した場合のロス）: 85
総トリムロス率 (対使用原反総材料): 7.08%


### モデルの改善と現実的な側面

  * **パターンの品質と網羅性**:
    このアプローチの成否は、事前に定義されるカッティングパターンの品質と網羅性に大きく依存する。最適なパターンが含まれていなければ、得られる解も真の最適解ではない可能性がある。実務では、過去のデータや経験、あるいは専用のパターン生成アルゴリズム（例えば、1本の原反からロスが最小になるように製品を切り出す問題を解く、これは一種のナップサック問題に似ている）を用いて、効率の良いパターンをリストアップする。
  * **トリムロス最小化との関係**:
    原反使用本数の最小化は、多くの場合、総トリムロスの最小化と密接に関連する。トリムロス自体を目的関数に含めることも考えられる。
  * **遅延列生成法 (Column Generation)**:
    大規模な問題や、パターン数が膨大になる場合には、Gilmore-Gomoryによって提案された遅延列生成法という高度なテクニックが用いられる。これは、最初少数のパターンで問題を解き、その結果（双対変数）を利用して、現在の解を最も改善する新しいパターンを動的に見つけ出し、モデルに追加していくという反復的な手法である。この手法は本講義の範囲を超えるが、カッティングストック問題の標準的な解法として知られている。
  * **現実世界の追加制約**:
    実際の工場では、以下のような追加の制約や考慮事項がある。
      * **カッターの制限**: 同時に切り出せる製品の最大数、最小幅など。
      * **パターンの切り替えコスト**: あるパターンから別のパターンに切り替える際に時間やコストがかかる場合、使用するパターンの種類数を制限したいことがある。
      * **注文の優先度**: 特定の製品を優先的に生産する必要がある。
      * **在庫考慮**: 完成品や中間製品の在庫を考慮した生産計画。
      * **2次元・3次元カッティング**: 板金やブロック材からの切り出し。これはさらに複雑になる。

本講義では、基本的な1次元カッティングストック問題を、パターンベースのアプローチで単純化して扱った。

---
#演習問題



##演習問題 1:

以下の条件でカッティングストック問題を解き、最小原反総数を求めよ。

  * 原反の幅: $L = 200 \text{cm}$
  * 製品データ:
      * 製品A: 幅 $70 \\text{cm}$, 需要 $15 \\text{本}$
      * 製品B: 幅 $50 \\text{cm}$, 需要 $20 \\text{本}$
      * 製品C: 幅 $30 \\text{cm}$, 需要 $25 \\text{本}$
  * 定義済みカッティングパターン (`[A本数, B本数, C本数]`):
    0.  `[2, 1, 0]` (70×2 + 50×1 = 190cm)
    1.  `[1, 2, 1]` (70×1 + 50×2 + 30×2=  0cm)
    2.  `[1, 1, 2]` (70×1 + 50×1 + 30×2 = 180cm)
    3.  `[0, 4, 0]` (50×4 = 200cm)
    4.  `[0, 2, 3]` (50×2 + 30×3 = 190cm)
    5.  `[0, 0, 6]` (30×6 = 180cm)
    6.  `[0, 1, 5]` (50×1 + 30×5 = 200cm)

##演習問題 2

演習問題1の条件のうち、製品の需要量のみが以下のように変更された場合の問題を解き、最小原反総数を求めよ。

  * 製品データ (需要量のみ変更):
      * 製品A: 幅 $70 \\text{cm}$, 需要 $8 \\text{本}$ (変更)
      * 製品B: 幅 $50 \\text{cm}$, 需要 $25 \\text{本}$ (変更)
      * 製品C: 幅 $30 \\text{cm}$, 需要 $15 \\text{本}$ (変更)
  * 原反幅、製品幅、カッティングパターンは演習問題1と同じ。


##演習問題 3

演習問題1の条件において、1本の原反から生じるトリムロスが
$0 \text{cm}$ を超えるパターンは使用してはならない、
という制約が追加された問題を解き、最小原反総数を求めよ。
